In [ ]:
import torch
from torch.nn import Module
import torch.nn.functional as F
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
import torch 
import numpy as np
import torch.nn as nn

class MLP_quantized(Module):
    def __init__(self):
        super(MLP_quantized, self).__init__()
        self.input   = qnn.QuantLinear(input_dim, 256, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.input_relu = qnn.QuantReLU(quant_type=QuantType.INT, bit_width=8, max_val=6)
        
        self.h1   = qnn.QuantLinear(256, 256, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.relu1 = qnn.QuantReLU(quant_type=QuantType.INT, bit_width=8, max_val=6)
        
        self.h2   = qnn.QuantLinear(256, 128, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)
        self.relu2 = qnn.QuantReLU(quant_type=QuantType.INT, bit_width=8, max_val=6)
        
        self.h3   = qnn.QuantLinear(128, output_dim, bias=False, 
                                     weight_quant_type=QuantType.INT, 
                                     weight_bit_width=8)

        self.output   = qnn.QuantSigmoid(quant_type=QuantType.FP, bit_width=8)

    def forward(self, x):
        x = self.input(x)
        x = self.input_relu(x)
        
        x = self.h1(x)
        x = self.relu1(x)
        
        x = self.h2(x)
        x = self.relu2(x)
        
        x = self.h3(x)
        return self.output(x)
    
#load model
model = MLP_quantized()
model.load_state_dict(torch.load("quantized_statedict.pt"))

#to use
#predict = model(input)